In [ ]:
import math
import uuid
import random
import pandas as pd
from datetime import datetime
from datetime import timedelta 

from src import tools_pandas
from src import tools_osm
from src import tools_str

NUM_RESULTS_PREVIEW = 3
DEBUG_MODE = True
PATH_DATA_STATISTICS = f"./data/input/statistics"

---
## Sources

### Age range by population
[Source: INE] https://www.ine.es/jaxi/Tabla.htm?path=/t20/e245/p05/a2014/l0/&file=00028001.px&L=0

In [ ]:
data_population = pd.read_csv(
    f"{PATH_DATA_STATISTICS}/madrid_population.csv"
)
data_population.head(NUM_RESULTS_PREVIEW) if DEBUG_MODE else None

### Case fatality rate COVID-19 by population

[Source: Ministerio de Sanidad (fecha: 28/03/2020)] https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov-China/documentos/Actualizacion_59_COVID-19.pdf

In [ ]:
data_mortality_rate = pd.read_csv(
    f"{PATH_DATA_STATISTICS}/mortality_rate_spain.csv", 
)
data_mortality_rate if DEBUG_MODE else None

### Random data generation  (Comunidad de Madrid)

#### Initial scenario

* **id**: Person ID (random)
* **lat**: Latitude (random)
* **lon**: Longitude (random)
* **location**: population
* **state**: people state (negative, suspect, infected) (random)
* **age**: Age (aleatorio)
* **date**: Date of sample collection

In [ ]:
INITIAL_STATE = "negative"
INITIAL_DATE = datetime.strptime("2020-03-31", "%Y-%m-%d")
MIN_PERCENT_POPULATION = 1.0
MAX_PERCENT_POPULATION = 2.0
MIN_RANGE_AGE = 16
MAX_RANGE_AGE = 100


data_people_initial_state = []

for index, row in data_population.iterrows():
    
    total_population = data_population.query(f"Poblacion == '{row['Poblacion']}'")["Total"].sum()
        
    population_name = tools_str.normalice_town_name([row["Poblacion"]])[0]
    
    item_norm = tools_str.normalize_name(population_name)
    
    geopanda = tools_osm.load_geopandas_file(item_norm, item_norm)

    points_population = []
    
    points_population.append(
        tools_osm.random_points_in_polygon(
            random.randint(
                math.ceil(total_population * (MIN_PERCENT_POPULATION/100)), 
                math.ceil(total_population * (MAX_PERCENT_POPULATION/100))), 
                geopanda["geometry"][0]
        )
    )
    

    for point in points_population:

        print(f'''
        Name: {population_name}
        Population: {total_population}
        Min. points: {math.ceil(total_population * (MIN_PERCENT_POPULATION/100))}
        Max. points: {math.ceil(total_population * (MAX_PERCENT_POPULATION/100))}
        Generated points: {len(point)}
        ''') if DEBUG_MODE else None

        for p in point:
            x, y = p.coords.xy
            data_people_initial_state.append(
                {
                    "id": uuid.uuid4(),
                    "lat": str(round(x[0], 6)),
                    "lon": str(round(y[0], 6)),
                    "location": population_name,
                    "state": INITIAL_STATE,
                    "age": random.randint(MIN_RANGE_AGE, MAX_RANGE_AGE),
                    "date": INITIAL_DATE
                }
            )

print(f"Num. points: {len(data_people_initial_state)}") if DEBUG_MODE else None

# Convert list to Dataframe
data_people_initial_state = pd.DataFrame(data_people_initial_state)

### Simulation (N days)

In [ ]:
NUM_DAYS_SIMULATION = 5

data_people_updated = data_people_initial_state.copy()
date_people_accumulated = data_people_initial_state.copy()

i = 0

while i < NUM_DAYS_SIMULATION:
    
    incremental_day = INITIAL_DATE + timedelta(days=i+1)
    
    data_people_updated['date'] = incremental_day
    
    print('Loading day', incremental_day, data_people_updated['id'].count()) if DEBUG_MODE else None
        
    for idx in data_people_updated.index:
        
        state_people_update = {
            "infected": random.choice(['infected', 'negative']),
            "suspected": random.choice(['suspected', 'infected']),
            "negative": random.choice(['negative', 'suspected', 'infected']),
            "recovered": "recovered"
        }
        
        last_state = data_people_updated.at[idx, 'state']
        new_state = state_people_update.get(last_state)
        
        if last_state == 'infected' and new_state == 'negative':
            data_people_updated.at[idx, 'state'] = 'recovered'
        else:
            data_people_updated.at[idx, 'state'] = new_state
        
    date_people_accumulated = date_people_accumulated.append(data_people_updated)
    
    i += 1
    
date_people_accumulated.to_csv(
    f"{PATH_DATA_STATISTICS}/people_data.csv", 
    encoding="utf-8",  
    float_format="%.2f", 
    header=True, 
    index=False,
)

---
## Statistical calculation - population level

### Normalize population name

To OpenStreetMap, we need to convert population name \{postal code\}-\{population\} to \{population\}, Madrid.
> Example: "28001-Acebeda, La" -> "La Acebeda, Madrid"

In [ ]:
data_population['location'] = tools_str.normalice_town_name(data_population['Poblacion'])
data_population.head(2) if DEBUG_MODE else None

### Aggregation by age range (60, 70, 80 or more)

In [ ]:
range_60 = data_population.iloc[:, 14:22].sum(axis=1)
range_70 = data_population.iloc[:, 16:22].sum(axis=1)
range_80 = data_population.iloc[:, 18:22].sum(axis=1)

data_population["percent_range_60"] = range_60 * 100 / data_population["Total"]
data_population["percent_range_70"] = range_70 * 100 / data_population["Total"]
data_population["percent_range_80"] = range_80 * 100 / data_population["Total"]


data_population = data_population.filter({
    'Poblacion',
    'Total',
    'percent_range_60',
    'percent_range_70',
    'percent_range_80',
    'location'
})

data_population if DEBUG_MODE else None

### Total cases

In [ ]:
total_cases = date_people_accumulated.groupby(["location", "date"])["state"].count().reset_index()
total_cases if DEBUG_MODE else None

In [ ]:
data_population_desnorm = pd.merge(data_population, total_cases, on="location", how="left")
data_population_desnorm if DEBUG_MODE else None

### Filter people by state

In [ ]:
dataset_filter_negative = date_people_accumulated.query("state == 'negative'")
dataset_filter_suspect = date_people_accumulated.query("state == 'suspected'")
dataset_filter_infected = date_people_accumulated.query("state == 'infected'")
dataset_filter_recovered = date_people_accumulated.query("state == 'recovered'")

### Aggregation by state and age range

In [ ]:
# Negative
agg_location_state_negative = (
    dataset_filter_negative.groupby(["location", "date"])["state"].count().reset_index()
)

agg_location_state_negative = agg_location_state_negative.rename(columns={'state': 'state_negative'})


# Enrichment dates by population
agg_location_states = pd.merge(
    data_population_desnorm, 
    agg_location_state_negative, 
    on=['location', 'date'], 
    how='left'
)


# Negative range 60 
agg_location_state_negative_range_60 = (
    dataset_filter_negative.query("age >= 60 and age <=69").groupby(["location", "date"])["state"].count().reset_index()
)

agg_location_state_negative_range_60 = agg_location_state_negative_range_60.rename(columns={'state': 'state_negative_range_60'})

# Enrichment dates by population
agg_location_states = pd.merge(
    agg_location_states, 
    agg_location_state_negative_range_60, 
    on=['location', 'date'], 
    how='left'
)

# Negative range 70
agg_location_state_negative_range_70 = (
    dataset_filter_negative.query("age >= 70 and age <=79").groupby(["location", "date"])["state"].count().reset_index()
)

agg_location_state_negative_range_70 = agg_location_state_negative_range_70.rename(columns={'state': 'state_negative_range_70'})

# Enrichment dates by population
agg_location_states = pd.merge(
    agg_location_states, 
    agg_location_state_negative_range_70, 
    on=['location', 'date'], 
    how='left'
)

# Negative range 80
agg_location_state_negative_range_80 = (
    dataset_filter_negative.query("age >= 80").groupby(["location", "date"])["state"].count().reset_index()
)

agg_location_state_negative_range_80 = agg_location_state_negative_range_80.rename(columns={'state': 'state_negative_range_80'})

# Enrichment dates by population
agg_location_states = pd.merge(
    agg_location_states, 
    agg_location_state_negative_range_80, 
    on=['location', 'date'], 
    how='left'
)


# Suspected
agg_location_state_suspect = (
    dataset_filter_suspect.groupby(["location", "date"])["state"].count().reset_index()
)

agg_location_state_suspect = agg_location_state_suspect.rename(columns={'state': 'state_suspected'})


# Enrichment dates by population
agg_location_states = pd.merge(
    agg_location_states, 
    agg_location_state_suspect, 
    on=['location', 'date'], 
    how='left'
)

# Suspected range 60 
agg_location_state_suspect_range_60 = (
    dataset_filter_suspect.query("age >= 60 and age <=69").groupby(["location", "date"])["state"].count().reset_index()
)


agg_location_state_suspect_range_60 = agg_location_state_suspect_range_60.rename(columns={'state': 'state_suspected_range_60'})

# Enrichment dates by population
agg_location_states = pd.merge(
    agg_location_states, 
    agg_location_state_suspect_range_60, 
    on=['location', 'date'], 
    how='left'
)

# Suspected range 70 
agg_location_state_suspect_range_70 = (
    dataset_filter_suspect.query("age >= 70 and age <=79").groupby(["location", "date"])["state"].count().reset_index()
)


agg_location_state_suspect_range_70 = agg_location_state_suspect_range_70.rename(columns={'state': 'state_suspected_range_70'})

# Enrichment dates by population
agg_location_states = pd.merge(
    agg_location_states, 
    agg_location_state_suspect_range_70, 
    on=['location', 'date'], 
    how='left'
)

# Suspected range 80
agg_location_state_suspect_range_80 = (
    dataset_filter_suspect.query("age >= 80").groupby(["location", "date"])["state"].count().reset_index()
)


agg_location_state_suspect_range_80 = agg_location_state_suspect_range_80.rename(columns={'state': 'state_suspected_range_80'})

# Enrichment dates by population
agg_location_states = pd.merge(
    agg_location_states, 
    agg_location_state_suspect_range_80, 
    on=['location', 'date'], 
    how='left'
)


# Infected
agg_location_state_infected = (
    dataset_filter_infected.groupby(["location", "date"])["state"].count().reset_index()
)

agg_location_state_infected = agg_location_state_infected.rename(columns={'state': 'state_infected'})

# Enrichment dates by population
agg_location_states = pd.merge(
    agg_location_states, 
    agg_location_state_infected, 
    on=['location', 'date'], 
    how='left'
)

# Infected range 60 
agg_location_state_infected_range_60 = (
    dataset_filter_infected.query("age >= 60 and age <=69").groupby(["location", "date"])["state"].count().reset_index()
)

agg_location_state_infected_range_60 = agg_location_state_infected_range_60.rename(columns={'state': 'state_infected_range_60'})

# Enrichment dates by population
agg_location_states = pd.merge(
    agg_location_states, 
    agg_location_state_infected_range_60, 
    on=['location', 'date'], 
    how='left'
)

# Infected range 70
agg_location_state_infected_range_70 = (
    dataset_filter_infected.query("age >= 70 and age <=79").groupby(["location", "date"])["state"].count().reset_index()
)

agg_location_state_infected_range_70 = agg_location_state_infected_range_70.rename(columns={'state': 'state_infected_range_70'})

# Enrichment dates by population
agg_location_states = pd.merge(
    agg_location_states, 
    agg_location_state_infected_range_70, 
    on=['location', 'date'], 
    how='left'
)


# Infected range 80
agg_location_state_infected_range_80 = (
    dataset_filter_infected.query("age >= 80").groupby(["location", "date"])["state"].count().reset_index()
)

agg_location_state_infected_range_80 = agg_location_state_infected_range_80.rename(columns={'state': 'state_infected_range_80'})

# Enrichment dates by population
agg_location_states = pd.merge(
    agg_location_states, 
    agg_location_state_infected_range_80, 
    on=['location', 'date'], 
    how='left'
)

# Recovered 
agg_location_state_recovered = (
    dataset_filter_recovered.groupby(["location", "date"])["state"].count().reset_index()
)

agg_location_state_recovered = agg_location_state_recovered.rename(columns={'state': 'state_recovered'})

# Enrichment dates by population
agg_location_states = pd.merge(
    agg_location_states, 
    agg_location_state_recovered, 
    on=['location', 'date'], 
    how='left'
)

# Recovered range 60 
agg_location_state_recovered_range_60 = (
    dataset_filter_recovered.query("age >= 60 and age <=69").groupby(["location", "date"])["state"].count().reset_index()
)

agg_location_state_recovered_range_60 = agg_location_state_recovered_range_60.rename(columns={'state': 'state_recovered_range_60'})

# Enrichment dates by population
agg_location_states = pd.merge(
    agg_location_states, 
    agg_location_state_recovered_range_60, 
    on=['location', 'date'], 
    how='left'
)

# Recovered range 70
agg_location_state_recovered_range_70 = (
    dataset_filter_recovered.query("age >= 70 and age <=79").groupby(["location", "date"])["state"].count().reset_index()
)

agg_location_state_recovered_range_70 = agg_location_state_recovered_range_70.rename(columns={'state': 'state_recovered_range_70'})

# Enrichment dates by population
agg_location_states = pd.merge(
    agg_location_states, 
    agg_location_state_recovered_range_70, 
    on=['location', 'date'], 
    how='left'
)

# Recovered range 80
agg_location_state_recovered_range_80 = (
    dataset_filter_recovered.query("age >= 80").groupby(["location", "date"])["state"].count().reset_index()
)

agg_location_state_recovered_range_80 = agg_location_state_recovered_range_80.rename(columns={'state': 'state_recovered_range_80'})

# Enrichment dates by population
agg_location_states = pd.merge(
    agg_location_states, 
    agg_location_state_recovered_range_80, 
    on=['location', 'date'], 
    how='left'
)

### Calculation about infected/suspected people

* **total_percent_infected**: Percentage of infected people 
* **total_percent_suspect**: percentage of suspected people 
* **total_percent_infected_range_60**: Percentage of infected people into range 60/69 ages.
* **total_percent_suspect_range_60**: Percentage of suspected people into range 60/69 ages.
* **susceptible_death_range_60**: Percentage of infected people likely to die (range 60/69 ages)
* **susceptible_death_range_70**: Percentage of infected people likely to die (range 70/79 ages)
* **susceptible_death_range_80_or_more**: Percentage of infected people likely to die (range 80 age or more)

In [ ]:
agg_location_states['total_percent_infected'] = agg_location_states['state_infected'] * 100 / agg_location_states['Total']
agg_location_states['total_percent_suspected'] = agg_location_states['state_suspected'] * 100 / agg_location_states['Total']
agg_location_states['total_percent_recovered'] = agg_location_states['state_recovered'] * 100 / agg_location_states['Total']
agg_location_states['total_percent_negative'] = agg_location_states['state_negative'] * 100 / agg_location_states['Total']

agg_location_states['total_percent_infected_range_60'] = agg_location_states['state_infected_range_60'] * 100 / agg_location_states['Total']
agg_location_states['total_percent_suspected_range_60'] = agg_location_states['state_suspected_range_60'] * 100 / agg_location_states['Total']
agg_location_states['total_percent_recovered_range_60'] = agg_location_states['state_recovered_range_60'] * 100 / agg_location_states['Total']
agg_location_states['total_percent_negative_range_60'] = agg_location_states['state_negative_range_60'] * 100 / agg_location_states['Total']


agg_location_states['total_percent_infected_range_70'] = agg_location_states['state_infected_range_70'] * 100 / agg_location_states['Total']
agg_location_states['total_percent_suspected_range_70'] = agg_location_states['state_suspected_range_70'] * 100 / agg_location_states['Total']
agg_location_states['total_percent_recovered_range_70'] = agg_location_states['state_recovered_range_70'] * 100 / agg_location_states['Total']
agg_location_states['total_percent_negative_range_70'] = agg_location_states['state_negative_range_70'] * 100 / agg_location_states['Total']


agg_location_states['total_percent_infected_range_80'] = agg_location_states['state_infected_range_80'] * 100 / agg_location_states['Total']
agg_location_states['total_percent_suspected_range_80'] = agg_location_states['state_suspected_range_80'] * 100 / agg_location_states['Total']
agg_location_states['total_percent_recovered_range_80'] = agg_location_states['state_recovered_range_80'] * 100 / agg_location_states['Total']
agg_location_states['total_percent_negative_range_80'] = agg_location_states['state_negative_range_80'] * 100 / agg_location_states['Total']



data_mortality_rate_range_60 = data_mortality_rate['60-69'].max()
data_mortality_rate_range_70 = data_mortality_rate['70-79'].max()
data_mortality_rate_range_80 = data_mortality_rate['80-89'].max()

agg_location_states['susceptible_death_range_60'] = agg_location_states['state_infected_range_60'] * data_mortality_rate_range_60 / 100
agg_location_states['susceptible_death_range_70'] = agg_location_states['state_infected_range_70'] * data_mortality_rate_range_70 / 100
agg_location_states['susceptible_death_range_80'] = agg_location_states['state_infected_range_80'] * data_mortality_rate_range_80 / 100


agg_location_states if DEBUG_MODE else None

### Rename columns and write results.

In [ ]:
columns = {
    'Poblacion': 'Population',
    'percent_range_60': '% Total range 60',
    'percent_range_70': '% Total range 70',
    'percent_range_80': '% Total range 80',
    'total_cases': 'Total cases',
    'state_negative': 'Negative',
    'state_negative_range_60': 'Negative range 60',
    'state_negative_range_70': 'Negative range 70',
    'state_suspected': 'Suspected',
    'state_suspected_range_60': 'Suspected range 60',
    'state_infected': 'Infected',
    'state_infected_range_60': 'Infected range 60',
    'state_infected_range_70': 'Infected range 70',
    'state_infected_range_80_or_more': 'Infected range 80 or more',
    
    
    'state_recovered': 'Recovered',
    'state_recovered_range_60': 'Recovered range 60',
    'state_recovered_range_70': 'Recovered range 70',
    
    'total_percent_infected': '% Total infected by population',
    'total_percent_suspected': '% Total supected by population',
    'total_percent_negative': '% Total negative by population',
    'total_percent_recovered': '% Total recovered by population',
    
    
    'total_percent_infected_range_60': '% Total infected range 60',
    'total_percent_suspected_range_60': '% Total supected rage 60',
    'total_percent_negative_range_60': '% Total negative range 60',
    'total_percent_recovered_range_60': '% Total recovered rage 60',
    

    'total_percent_infected_range_70': '% Total infected range 70',
    'total_percent_suspected_range_70': '% Total supected rage 70',
    'total_percent_negative_range_70': '% Total negative range 70',
    'total_percent_recovered_range_70': '% Total recovered rage 70',
    

    'total_percent_infected_range_80': '% Total infected range 80',
    'total_percent_suspected_range_80': '% Total supected rage 80',
    'total_percent_negative_range_80': '% Total negative range 80',
    'total_percent_recovered_range_80': '% Total recovered rage 80',
    
    'susceptible_death_range_60': 'Susceptible death 60/69 age',
    'susceptible_death_range_70': 'Susceptible death 70/79 age',
    'susceptible_death_range_80': 'Susceptible death 80 or more age',
}

agg_location_states = pd.DataFrame(agg_location_states.rename(columns=columns)).fillna(0)


agg_location_states.to_csv(
    f"{PATH_DATA_STATISTICS}/all_stats_madrid.csv", encoding="utf-8",  float_format="%.2f", header=True, index=False,
)

agg_location_states.head(NUM_RESULTS_PREVIEW) if DEBUG_MODE else None